# Three Tanks System: Comparison of SysID Models

In [1]:
import warnings
warnings.filterwarnings("ignore")

# For solving ODEs
from scipy.integrate import solve_ivp

# Libraries for MKCVA and CVA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from matplotlib import cm, colors
from scipy.linalg import cholesky
import matplotlib.pyplot as plt
import statsmodels.api as sm
from time import time
import pandas as pd
import numpy as np
import cyipopt
import pickle

# Libraries for LSTM
import gc
import keras
import tensorflow as tf
from keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import Model,load_model

## Declare all classes

In [8]:
class CVA:
    def __init__(self, verbose=None):
        if verbose == None:
            self.verbose = 0
        else:
            self.verbose = verbose
    
    def identify(self, Z_train, UI, YI, n_states=None):
        
        start = time()
        self.UI = UI  # Column indices of input vars 
        self.YI = YI  # Column indices of output vars
        N = Z_train.shape[0]

        # Perform Standard Scaling on raw data [u y]
        sc_raw = StandardScaler()
        Z_train_sc = sc_raw.fit_transform(Z_train)
        y_train = Z_train_sc[:, YI]
        self.sc_raw = sc_raw

        # Calculate the suggested no. of lags on the KPCA scores
        _, ci = sm.tsa.acf(np.sum(y_train ** 2, axis=1), alpha=0.05)
        self.n_lags = np.argwhere(ci[:,0] < 0)[0][0]
        p = f = self.n_lags
        
        # Create Hankel matrices: Yp and Yf from KPCA scores
        Yp, Yf = [], []
        for k in np.arange(N-p-f):
            Yp.append(np.flip(Z_train_sc[k:k+p, :].reshape(-1, 1)))

        for k in np.arange(1, N-p-f+1):
            Yf.append(Z_train_sc[k+p:k+p+f, YI].reshape(-1, 1))

        Yp = np.transpose(np.hstack(Yp))
        Yf = np.transpose(np.hstack(Yf))
        Np = Yp.shape[0]

        # Standardize the Hankel matrices
        sc_p = StandardScaler()
        sc_f = StandardScaler()
        Yp_scaled = sc_p.fit_transform(Yp)
        Yf_scaled = sc_f.fit_transform(Yf)
        self.sc_p = sc_p

        # Perform CCA
        Epp = cholesky(np.dot(Yp_scaled.T, Yp_scaled))  # Past Cholesky matrix
        Eff = cholesky(np.dot(Yf_scaled.T, Yf_scaled))  # Future Cholesky matrix
        Efp = np.dot(Yf_scaled.T, Yp_scaled)            # Cross-covariance matrix
        H = np.linalg.inv(Eff.T) @ Efp @ np.linalg.inv(Epp)

        U, S, V = np.linalg.svd(H)

        # Calculate the suggested no. of states via knee of SV plot
        if n_states == None:
            n_states = np.minimum(2 + np.argmax(np.diff(np.diff(S))), 10)
        self.n_states = n_states

        if self.verbose:
            print(f'No. of lags: {self.n_lags}')
            plt.plot(np.arange(15)+1, S[:15], 'b.--')
            plt.scatter(n_states, S[n_states-1], c='r')
            plt.title('Singular Values plot')
            print(f'No. of states: {n_states}')
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Calculate the state vectors, X
        Vn = np.transpose(V[:n_states, :])
        Jn = np.dot(Vn.T, np.linalg.inv(Epp.T))
        X = Jn @ Yp_scaled.T
        self.Jn = Jn
        
        # Solve for A, B, C, D, K
        M = X.shape[1]
        tk = np.transpose(Z_train_sc[p-1:p+M-1, self.UI])
        yk = np.transpose(y_train[p:p+M, :])

        CD = yk[:,:(M-1)] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                      tk[:,:(M-1)])))
        C = CD[:len(YI), :n_states]          # Output matrix
        D = CD[:len(YI), n_states:]          # Feedthrough matrix

        E = yk[:,:(M-1)] - C @ X[:,:(M-1)] - D @ tk[:,:(M-1)]
        ABK = X[:,1:M] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                   tk[:,:(M-1)], E)))
        A = ABK[:,:n_states]                 # State transition matrix
        B = ABK[:,n_states:(n_states+len(UI))]  # Input matrix
        K = ABK[:,(n_states+len(UI)):]          # Kalman gain
        
        self.ident_time = time() - start     # Time elapsed for identification
        
        self.A, self.B, self.C, self.D, self.K = A, B, C, D, K
        self.X = X
       
    def init_sim(self, Z_test_sc):
        # Calculate initial state x(0) using CVA projection matrix
        yp = np.flip(Z_test_sc[:self.n_lags, :].reshape(1, -1))
        Yp_scaled = self.sc_p.transform(yp)
        x0 = self.Jn @ Yp_scaled.T
        return x0
    
    def simulate(self, Z_test):
        start = time()
        Nt = Z_test.shape[0]
        Z_test_sc = self.sc_raw.transform(Z_test)
        x_pred = np.zeros((self.n_states, Nt - self.n_lags + 1)) 
        y_pred = np.zeros((len(self.YI), Nt - self.n_lags + 1))
        x0 = self.init_sim(Z_test_sc)
        u0 = Z_test_sc[0, self.UI].reshape(-1, 1)
        y0 = self.C @ x0 + self.D @ u0
        x_pred[:, 0] = x0.ravel()
        y_pred[:, 0] = y0.ravel()
        exit_code = 0
        
        for j in np.arange(1, y_pred.shape[1]):
            uk = Z_test_sc[j+self.n_lags-1, self.UI].reshape(-1, 1)
            xk_1 = x_pred[:, j-1].reshape(-1, 1)
            xk = self.A @ xk_1 + self.B @ uk
            yk = self.C @ xk + self.D @ uk
            x_pred[:, j] = xk.ravel()
            y_pred[:, j] = yk.ravel()
            if (np.abs(yk) > 1e3).any():
                exit_code = -1
        
        x_pred = np.transpose(x_pred)
        y_pred = np.transpose(y_pred)
        y_pred = (y_pred * self.sc_raw.scale_[self.YI]) + \
                           self.sc_raw.mean_[self.YI]
        self.sim_time = time() - start
        
        return x_pred, y_pred, exit_code
    
    def R2_score(self, Z_test, y_pred):
        r2 = np.zeros(len(self.YI))
        for k in range(len(self.YI)):
            y_true = Z_test[self.n_lags-1:, self.YI[k]]
            r2[k] = 1 - np.sum((y_true - y_pred[:, k]) ** 2) \
                    / np.sum((y_true - np.mean(y_true)) ** 2)   
        return r2
    
    def display(self):
        print(f'No. of lags: {self.n_lags}')
        print(f'No. of states: {self.n_states}')
        print(f'Indices of u: {self.UI}')
        print(f'Indices of y: {self.YI}')
        print('State-space matrices:')
        print(self.A)
        print(self.B)
        print(self.C)
        print(self.D)
        print(self.K)
        
class KPCA:
    def kernel_func(self, x1, x2):
        # x1 size: [no. of samples x no. of features]
        # x2 size: [no. of samples x no. of features]
        
        D = np.sum((x1 / self.kw) ** 2, axis=1, keepdims=True) \
            + np.sum((x2 / self.kw).T ** 2, axis=0, keepdims=True) \
            - 2 * np.tensordot(x1 / self.kw**2, x2.T, axes=1)
        L = np.tensordot(x1, x2.T, axes=1) + 1
        return self.w * L + (1 - self.w) * np.exp(-D)
        
    def fit_transform(self, X, kw, w, n_comp=None):
        # Compute kernel matrix
        self.X = X
        self.kw = kw
        self.w = w
        self.N = len(self.X)
        
        # Calculate reduced kernel matrix with k-medoids clustering 
        dist = pairwise_distances(self.X)
        dm = kmedoids.KMedoids(method='fasterpam', 
                               n_clusters=int(0.95*self.N), 
                               random_state=0).fit(dist)
        self.X_red = X[dm.medoid_indices_, :]
        self.N_red = len(dm.medoid_indices_)
        self.medoid_indices = dm.medoid_indices_
        self.K = self.kernel_func(self.X_red, self.X_red)

        # Center the kernel matrix
        self.U = np.ones((self.N_red, self.N_red)) / self.N_red
        Kc = self.K - self.U @ self.K - self.K @ self.U + self.U @ self.K @ self.U

        # Perform eigenvalue decomposition
        eigvals, eigvecs = np.linalg.eigh(Kc / self.N_red)

        # Ensure the eigenvalues are sorted in decreasing order
        ind = (-eigvals).argsort()
        eigvals = eigvals[ind]
        eigvecs = eigvecs[:,ind]
        
        if n_comp == None:
            # Get eigenvalues using CPV = 99%
            CPV = np.cumsum(eigvals) / np.sum(eigvals)
            self.n_comp = np.argwhere(CPV > 0.99)[0][0]
            self.n_comp = np.minimum(self.n_comp, 15)
        else:
            self.n_comp = n_comp
        
        self.CPV = CPV
        
        # Compute the projection matrix
        self.P = eigvecs[:,:self.n_comp] @ np.diag(eigvals[:self.n_comp] ** -0.5)
        
        # Project the training data X via the reduced centered kernel matrix
        scores = self.transform(self.X)
        return scores
    
    def transform(self, Xt):
        if len(Xt.shape) == 1:
            Xt = Xt.reshape(1, -1)
        
        Kt = np.transpose(self.kernel_func(self.X_red, Xt))
        Ut = np.ones((Xt.shape[0], self.N_red)) / self.N_red
        Kct = Kt - Ut @ self.K - Kt @ self.U + Ut @ self.K @ self.U
        scores = Kct @ self.P
        return scores
    
class MKCVA:
    def __init__(self, verbose=None):
        if verbose == None:
            self.verbose = 0
        else:
            self.verbose = verbose
    
    def identify(self, Z_train, UI, YI, kw=None, w=None, n_states=None):
        
        start = time()
        self.UI = UI  # Column indices of input vars 
        self.YI = YI  # Column indices of output vars
        N = Z_train.shape[0]

        # Perform Standard Scaling on raw data [u y]
        sc_raw = StandardScaler()
        Z_train_sc = sc_raw.fit_transform(Z_train)
        y_train = Z_train_sc[:, YI]
        self.sc_raw = sc_raw

        # Perform KPCA on scaled [u(k), y(k-1)] data
        
        Z_kpca = np.hstack((Z_train_sc[1:, self.UI], 
                            Z_train_sc[:-1, self.YI]))
        
        kpca_uy = KPCA()
        kpca_score_uy = kpca_uy.fit_transform(Z_kpca, kw, w)
        self.kpca_uy = kpca_uy
        
        # Perform KPCA on scaled [y] data
        kpca_y = KPCA()
        kw = kpca_uy.kw * (len(YI) / Z_train.shape[1])
        kpca_score_y = kpca_y.fit_transform(Z_train_sc[:,YI], kw[YI], w)
        N_uy, N_y = kpca_uy.n_comp, kpca_y.n_comp

        # Calculate the suggested no. of lags on the KPCA scores
        _, ci = sm.tsa.acf(np.sum(kpca_score_y ** 2, axis=1), alpha=0.05)
        self.n_lags = np.argwhere(ci[:,0] < 0)[0][0]
        p = f = self.n_lags
        
        # Create Hankel matrices: Yp and Yf from KPCA scores
        Yp, Yf, Yp_all = [], [], []
        for k in np.arange(N-p-f):
            Yp.append(np.flip(kpca_score_uy[k:k+p, :].reshape(-1, 1)))

        for k in np.arange(1, N-p-f+1):
            Yf.append(kpca_score_y[k+p:k+p+f, :].reshape(-1, 1))

        Yp = np.transpose(np.hstack(Yp))
        Yf = np.transpose(np.hstack(Yf))
        Np = Yp.shape[0]

        # Standardize the Hankel matrices
        sc_p = StandardScaler()
        sc_f = StandardScaler()
        Yp_scaled = sc_p.fit_transform(Yp)
        Yf_scaled = sc_f.fit_transform(Yf)
        self.sc_p = sc_p

        # Perform CCA
        Epp = cholesky(np.dot(Yp_scaled.T, Yp_scaled))  # Past Cholesky matrix
        Eff = cholesky(np.dot(Yf_scaled.T, Yf_scaled))  # Future Cholesky matrix
        Efp = np.dot(Yf_scaled.T, Yp_scaled)            # Cross-covariance matrix
        H = np.linalg.inv(Eff.T) @ Efp @ np.linalg.inv(Epp)

        U, S, V = np.linalg.svd(H)

        # Calculate the suggested no. of states via knee of SV plot
        if n_states == None:
            n_states = np.minimum(2 + np.argmax(np.diff(np.diff(S))), 10)
        self.n_states = n_states

        if self.verbose:
            print(f'No. of KPCs on [u y]: {N_uy}')
            print(f'No. of KPCs on [y]:   {N_y}')
            print(f'No. of lags: {self.n_lags}')
            plt.figure(figsize=(12, 3))
            plt.subplot(131)
            plt.title('CPV Plot for KPCA on [u y]')
            plt.plot(np.arange(len(kpca_uy.CPV))+1, kpca_uy.CPV, 'b.--')
            plt.scatter(kpca_uy.n_comp, kpca_uy.CPV[kpca_uy.n_comp-1], c='r')
            plt.xlim([0, 30])
            plt.grid()
            plt.subplot(132)
            plt.title('CPV Plot for KPCA on [y]')
            plt.plot(np.arange(len(kpca_y.CPV))+1, kpca_y.CPV, 'b.--')
            plt.scatter(kpca_y.n_comp, kpca_y.CPV[kpca_y.n_comp-1], c='r')
            plt.xlim([0, 30])
            plt.grid()
            plt.subplot(133)
            plt.plot(np.arange(15)+1, S[:15], 'b.--')
            plt.scatter(n_states, S[n_states-1], c='r')
            plt.title('Singular Values plot')
            print(f'No. of states: {n_states}')
            plt.grid()
            plt.tight_layout()
            plt.show()

        # Calculate the state vectors, X
        Vn = np.transpose(V[:n_states, :])
        Jn = np.dot(Vn.T, np.linalg.inv(Epp.T))
        X = Jn @ Yp_scaled.T
        self.Jn = Jn
        
        # Solve for A, B, C, D, K
        M = X.shape[1]
        tk = np.transpose(kpca_score_uy[p-1:p+M-1, :])
        yk = np.transpose(y_train[p:p+M, :])

        CD = yk[:,:(M-1)] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                      tk[:,:(M-1)])))
        C = CD[:len(YI), :n_states]          # Output matrix
        D = CD[:len(YI), n_states:]          # Feedthrough matrix

        E = yk[:,:(M-1)] - C @ X[:,:(M-1)] - D @ tk[:,:(M-1)]
        ABK = X[:,1:M] @ np.linalg.pinv(np.vstack((X[:,:(M-1)], 
                                                   tk[:,:(M-1)], E)))
        A = ABK[:,:n_states]                 # State transition matrix
        B = ABK[:,n_states:(n_states+N_uy)]  # Input matrix
        K = ABK[:,(n_states+N_uy):]          # Kalman gain
        
        self.ident_time = time() - start     # Time elapsed for identification
        
        self.A, self.B, self.C, self.D, self.K = A, B, C, D, K
        self.X = X
       
    def init_sim(self, Z_test_sc):
        # Calculate initial state x(0) using CVA projection matrix
        tk = self.kpca_uy.transform(Z_test_sc)
        tk_p = np.flip(tk[:self.n_lags, :].reshape(1, -1))
        Yp_scaled = self.sc_p.transform(tk_p)
        x0 = self.Jn @ Yp_scaled.T
        t0 = tk[self.n_lags, :].reshape(-1, 1)
        return x0, t0
    
    def simulate(self, Z_test):
        start = time()
        Nt = Z_test.shape[0]
        Z_test_sc = self.sc_raw.transform(Z_test)
        x_pred = np.zeros((self.n_states, Nt - self.n_lags + 1)) 
        y_pred = np.zeros((len(self.YI), Nt - self.n_lags + 1))
        x0, t0 = self.init_sim(Z_test_sc)
        y0 = self.C @ x0 + self.D @ t0
        x_pred[:, 0] = x0.ravel()
        y_pred[:, 0] = y0.ravel()
        exit_code = 0
        
        for j in np.arange(1, y_pred.shape[1]):
            zk = np.hstack((Z_test_sc[j+self.n_lags-1, self.UI], 
                            y_pred[:, j-1]))
            tk = self.kpca_uy.transform(zk.reshape(1, -1))
            xk_1 = x_pred[:, j-1].reshape(-1, 1)
            xk = self.A @ xk_1 + self.B @ tk.T
            yk = self.C @ xk + self.D @ tk.T
            x_pred[:, j] = xk.ravel()
            y_pred[:, j] = yk.ravel()
            if (np.abs(yk) > 1e3).any():
                exit_code = -1
        
        x_pred = np.transpose(x_pred)
        y_pred = np.transpose(y_pred)
        y_pred = (y_pred * self.sc_raw.scale_[self.YI]) + \
                           self.sc_raw.mean_[self.YI]
        self.sim_time = time() - start
        
        return x_pred, y_pred, exit_code
    
    def R2_score(self, Z_test, y_pred):
        r2 = np.zeros(len(self.YI))
        for k in range(len(self.YI)):
            y_true = Z_test[self.n_lags-1:, self.YI[k]]
            r2[k] = 1 - np.sum((y_true - y_pred[:, k]) ** 2) \
                    / np.sum((y_true - np.mean(y_true)) ** 2)   
        return r2
    
    def display(self):
        print(f'No. of medoids: {self.kpca_uy.N_red}')
        print(f'No. of KPCs on [u y]: {self.kpca_uy.n_comp}')
        print(f'No. of lags: {self.n_lags}')
        print(f'No. of states: {self.n_states}')
        print(f'Indices of u: {self.UI}')
        print(f'Indices of y: {self.YI}')
        print('State-space matrices:')
        print(self.A)
        print(self.B)
        print(self.C)
        print(self.D)
        print(self.K)

## Load all models

In [9]:
cva_mdl = pickle.load(open('tanks_cva_sys.pkl','rb'))
mkcva_mdl = pickle.load(open('tanks_mkcva_sys.pkl','rb'))
lstm_mdl = load_model('tanks_lstm.keras')

look_back = 23
YI, UI = np.array([2, 3, 4]), np.array([0, 1])
input_spec = tf.TensorSpec([None, look_back, len(UI)+len(YI)], dtype=tf.float32)
lstm_func = tf.function(lstm_mdl).get_concrete_function(input_spec)

# Use Tensorflow's XLA (Accelerated Linear Algebra) for faster inference
@tf.function(jit_compile=True)
def lstm_predict(x):
    return lstm_func(tf.cast(x, tf.float32))

# Standardscaler params of original Training Data (for LSTM use only)
scale_ = np.array([8.46453479e-06, 7.29156132e-06, 4.23092551e-02, 4.05435287e-02, 3.80664313e-02])
mean_ = np.array([3.08757950e-05, 2.85512014e-05, 2.16306891e-01, 3.47345648e-01, 2.80801000e-01])

## Methods for evaluating models

In [10]:
def prepare_data(tanks_df):
    noise_var = np.array([0, 0, 0.02, 0.02, 0.02])
    v_name = ['u1', 'u2', 'x1', 'x2', 'x3']
    res = list()
    for j in range(len(noise_var)):
        data = tanks_df[v_name[j]].values
        data += (np.random.rand(len(data))-0.5)*noise_var[j]
        res.append(data)
    
    return np.transpose(np.vstack(res))

def eval_pred(Z, y_pred):
    r2 = np.zeros(len(YI))
    for k in range(len(YI)):
        y_true = Z[-len(y_pred):, YI[k]]
        r2[k] = 1 - np.sum((y_true - y_pred[:, k]) ** 2) \
                / np.sum((y_true - np.mean(y_true)) ** 2)
    
    return r2
    
def eval_CVA_simulate(Z):
    sim_time = time()
    x_pred, y_pred, exit_code = cva_mdl.simulate(Z)
    sim_time = time() - sim_time
    r2 = eval_pred(Z, y_pred)
    return r2, sim_time

def eval_MKCVA_simulate(Z):
    sim_time = time()
    x_pred, y_pred, exit_code = mkcva_mdl.simulate(Z)
    sim_time = time() - sim_time
    r2 = eval_pred(Z, y_pred)
    return r2, sim_time

def eval_LSTM_simulate(Z):
    start = time()
    uy0 = Z[:look_back, :]
    u = Z[look_back-1:, UI]
    
    K.clear_session()
    
    N = u.shape[0]
    y_pred = np.zeros((N, len(YI)))
    uy = (uy0 - mean_) / scale_
    u_ = (u - mean_[UI]) / scale_[UI]
    y_pred[0, :] = uy[-1, YI]
    
    for j in np.arange(1, N):
        uy = np.hstack((np.vstack((uy[1:, UI], 
                                   u_[np.minimum(j, N), :])), 
                        uy[:, YI]))
        y = lstm_predict(uy[np.newaxis, :, :])
        y_pred[j, :] = y[:, -1, :]
        uy = np.hstack((uy[:, UI],
                        np.vstack((uy[1:, YI], y[:, -1, :]))))
    
    y_pred = y_pred * scale_[YI] + mean_[YI]
    sim_time = time() - start
    r2 = eval_pred(Z, y_pred)
    return r2, sim_time

## Define ODE and simulate

In [23]:
def solve_model(u1, u2):
    def dxdt(t, x):
        pos = np.argwhere(t >= i_data)[-1]
        return [1/AT*(-AC*ac*np.sign(x[0]-x[2])*np.sqrt(2*g*np.abs(x[0]-x[2])) + u1[pos][0] - \
                      AO*a0*np.sqrt(2*g*x[0])),
                1/AT*(AC*ac*np.sign(x[2]-x[1])*np.sqrt(2*g*np.abs(x[2]-x[1])) + u2[pos][0]) - \
                      AO*a0*np.sqrt(2*g*x[1]),
                1/AT*(AC*ac*np.sign(x[0]-x[2])*np.sqrt(2*g*np.abs(x[0]-x[2]))\
                     -AC*ac*np.sign(x[2]-x[1])*np.sqrt(2*g*np.abs(x[2]-x[1])))]

    sol = solve_ivp(dxdt, [t_span[0], t_span[-1]], x0, t_eval=t_span)

    u1_t = u1[[np.argwhere(j >= i_data)[-1] for j in sol.t]].ravel()
    u2_t = u2[[np.argwhere(j >= i_data)[-1] for j in sol.t]].ravel()
    
    tanks_df = pd.DataFrame({'x1': sol.y[0], 'x2': sol.y[1] , 'x3':sol.y[2], 'u1':u1_t, 'u2':u2_t})
    
    return tanks_df

# Parameters
hmax = 0.62    # m, max water level
AT = 154e-4    # m^2, tank cross-sectional area
AO = 0.5e-4    # m^2, cross-sectional area of valve
AC = 0.5e-4    # m^2, cross-sectional area of connecting pipes
a0 = 0.56      # valve coefficient, outlet
ac = 0.48      # valve coefficient, connecting pipes
g = 9.81       # m/s^2
qmax = 100e-6  # m^3/s, max flow rate in

# Sampling: 1 sample every 10 sec
t_span = np.linspace(0, 5000, 501)

# Nominal conditions chosen:
x0 = np.array([0.1961, 0.3243, 0.26])  # m, tank liquid levels
u0 = 28e-6                             # m^3/s, inlet flow rates
    
i_data = np.arange(t_span[0], t_span[-1], 240) # start, last, increment

for ex in np.array([1, 1.25, 1.5]): # 1=interp, 1.25=extrap, 1.5=extrap
    
    cva_r2, cva_time = [], []
    mkcva_r2, mkcva_time = [], []
    lstm_r2, lstm_time = [], []
    
    for trial in range(50):
        u1 = (np.random.rand(i_data.shape[0])-0.5)*30e-6*ex + u0
        u2 = (np.random.rand(i_data.shape[0])-0.5)*30e-6*ex + u0

        u1[0] = u0
        u2[0] = u0

        sim_time = time()
        tanks_df = solve_model(u1, u2)
        sim_time = time() - sim_time
        #print(f'Trial {trial}: [{sim_time:.2f} sec]')

        Z = prepare_data(tanks_df)

        r2, sim_time = eval_CVA_simulate(Z)
        cva_r2.append(np.mean(r2))
        cva_time.append(sim_time)

        r2, sim_time = eval_MKCVA_simulate(Z)
        mkcva_r2.append(np.mean(r2))
        mkcva_time.append(sim_time)

        r2, sim_time = eval_LSTM_simulate(Z)
        lstm_r2.append(np.mean(r2))
        lstm_time.append(sim_time)

    df = pd.DataFrame(np.column_stack([cva_r2, cva_time, mkcva_r2, mkcva_time, lstm_r2, lstm_time]),
                      columns=['CVA-R2', 'CVA-Time', 'MKCVA-R2', 
                               'MKCVA-Time', 'LSTM-R2', 'LSTM-Time'])
    print(df.head()) 
    df.to_csv(f'compare_sysID_{ex}.csv', index=False)

     CVA-R2  CVA-Time  MKCVA-R2  MKCVA-Time   LSTM-R2  LSTM-Time
0  0.837570  0.129702  0.955954    0.411928  0.869579   1.843975
1  0.798684  0.007977  0.924121    0.253191  0.836920   0.541500
2  0.846883  0.015621  0.929273    0.240113  0.954486   0.504445
3  0.868405  0.000000  0.940339    0.228265  0.915446   0.481591
4  0.858906  0.007940  0.941682    0.205080  0.934940   0.490880
     CVA-R2  CVA-Time  MKCVA-R2  MKCVA-Time   LSTM-R2  LSTM-Time
0  0.898426  0.010661  0.966030    0.275975  0.954189   0.684142
1  0.809189  0.014693  0.963045    0.231455  0.939346   0.503414
2  0.862551  0.011891  0.940900    0.211015  0.931574   0.473097
3  0.788401  0.015625  0.896061    0.219600  0.557976   0.485962
4  0.855356  0.015620  0.938184    0.207502  0.890697   0.457860
     CVA-R2  CVA-Time  MKCVA-R2  MKCVA-Time   LSTM-R2  LSTM-Time
0  0.837252  0.010934  0.910353    0.235167  0.630782   0.575322
1  0.722750  0.015039  0.859748    0.218580  0.180251   0.562115
2  0.619598  0.013152  0.